In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from IPython.display import Markdown, display

from dotenv import load_dotenv

load_dotenv('/workspaces/ml-learning/src/airtable-chatbot/.env', override=True)

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

https://llamahub.ai/l/airtable?from=all

Base ID: The path in the URL that begins with app  appfjm76R87oVG3BP  
Table ID: The path in the URL that begins with tbl tblkBhmgAFBRGmNRL  

https://support.airtable.com/docs/creating-personal-access-tokens  




In [ ]:
!pip install pyairtable

In [5]:
from llama_index import download_loader
import os

AirtableReader = download_loader('AirtableReader')

AIRTABLE_TOKEN=os.environ['AIRTABLE_TOKEN']
AIRTABLE_BASE_ID=os.environ['AIRTABLE_BASE_ID']
AIRTABLE_TABLE_ID=os.environ['AIRTABLE_TABLE_ID']

reader = AirtableReader(AIRTABLE_TOKEN)
documents = reader.load_data(table_id=AIRTABLE_TABLE_ID,base_id=AIRTABLE_BASE_ID)

In [17]:
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.llms import OpenAI


index = VectorStoreIndex.from_documents(documents)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [24]:

from llama_index import ServiceContext
llm = OpenAI(model="gpt-4", temperature=0)
service_context = ServiceContext.from_defaults(
    llm=llm,
)

query_engine = index.as_query_engine(service_context=service_context)

response = query_engine.query('What is Chamira\'s project?')

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [25]:
response.response

"Chamira's project is called Minerva. It is aimed at revolutionizing the educational space by leveraging LLM technology through an education-first architecture. The goal is to improve upon the current system, which Chamira believes is ineffective due to the ratio of students to teachers in classrooms. The project is inspired by the effectiveness of Open-Domain Question Answering as revealed by ChatGPT, but aims to structure it in a way that can make a significant impact in education."

In [11]:
response.source_nodes[0].node

TextNode(id_='5b000a73-06a2-480e-b835-58582f8a50e2', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ca4a5898-f09e-45c3-93ad-ed2d076fc889', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='0aacf5616a0bb588455d93d76345cca4380eff89d0ce59a0adfd3b238698234e'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='82c649be-99b5-4041-9181-895039e09f1f', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='6b1510515a9d3b93f4eb42c6da6a19014b219a804f42ff7e5b99bfbe5b86fb4e'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='80cec339-7588-462e-ac04-2f89be679fa2', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='2d711e7560c5d10d44b977f0ebbe83dbf61083dd6c82abf71dece4f9f8f55bd0')}, text='": \'https://www.linkedin.com/in/shaemckenna/\', \'What are your areas of expertise you have (select max 4 please)\': [\'AI Engineer\', \'Product management\', \'AI / ML special

In [21]:
from pprint import pprint
pprint(response.source_nodes[0].node.text)

('": \'https://www.linkedin.com/in/shaemckenna/\', \'What are your areas of '
 "expertise you have (select max 4 please)': ['AI Engineer', 'Product "
 "management', 'AI / ML specialist researcher'], 'What will you build': "
 '"I am planning on launching a startup in the edtech space. I always thought '
 'classrooms were a really ineffective way of teaching, one teacher for 25+ '
 "students just doesn't seem effective no matter how good a teacher may be. "
 'ChatGPT revealed the effectiveness of Open-Domain Question Answering '
 'however, it is still too unstructured to make a true dent in the educational '
 'space. This is where I see project Minerva coming into play. I aim to '
 'leverage LLM technology through an education-first architecture, to '
 'revolutionize how students learn around the world.')


In [ ]:
"""Airtable reader."""
from typing import List

from llama_index.readers.base import BaseReader
from llama_index.readers.schema.base import Document


class AirtableReader(BaseReader):
    """Airtable reader. Reads data from a table in a base.

    Args:
        api_key (str): Airtable API key.
    """

    def __init__(self, api_key: str) -> None:
        """Initialize Airtable reader."""

        self.api_key = api_key

    def load_data(self, base_id: str, table_id: str) -> List[Document]:
        """Load data from a table in a base

        Args:
            table_id (str): Table ID.
            base_id (str): Base ID.
        Returns:
            List[Document]: List of documents.
        """
        from pyairtable import Table

        table = Table(self.api_key, base_id, table_id)
        all_records = table.all()
        return [Document(text=f"{all_records}", extra_info={})]